In [1]:
from utils import *
from sympy import symbols, trigsimp, lambdify, diff, init_printing, eye
from numpy import linalg, cos, sin, arctan2, log
init_printing(use_unicode=True)

In [2]:
# Joint parameters declaration
q1, q2, q3, q4, q5 = symbols("q1 q2 q3 q4 q5", real=True)

# Guesses for link lengths
l1, l2, l3, l4, l5 = 1, 1, 1, 1, 1

In [3]:
# Forward Kinematics
H = Rx(q1)*Tx(l1)*Ry(q2)*Tz(l2)*Rz(q3)*Tz(l3)*Rx(q4)*Tz(l4)*Rz(q5)*Tz(l5)
H = trigsimp(H)
H

⎡                                (sin(q₂)⋅sin(q₄) - sin(q₃)⋅cos(q₂)⋅cos(q₄))⋅s
⎢                                                                             
⎢((-sin(q₁)⋅sin(q₂)⋅sin(q₃) + cos(q₁)⋅cos(q₃))⋅cos(q₄) - sin(q₁)⋅sin(q₄)⋅cos(q
⎢                                                                             
⎢((sin(q₁)⋅cos(q₃) + sin(q₂)⋅sin(q₃)⋅cos(q₁))⋅cos(q₄) + sin(q₄)⋅cos(q₁)⋅cos(q₂
⎢                                                                             
⎣                                                                      0      

in(q₅) + cos(q₂)⋅cos(q₃)⋅cos(q₅)                                              
                                                                              
₂))⋅sin(q₅) + (sin(q₁)⋅sin(q₂)⋅cos(q₃) + sin(q₃)⋅cos(q₁))⋅cos(q₅)  ((-sin(q₁)⋅
                                                                              
))⋅sin(q₅) + (sin(q₁)⋅sin(q₃) - sin(q₂)⋅cos(q₁)⋅cos(q₃))⋅cos(q₅)   ((sin(q₁)⋅c
                                                   

In [ ]:
# Rotation Matrix
R = H[0:3, :-1]
R

In [ ]:
# Translational Matrix
T = H[0:3, -1]
T

In [ ]:
# CALCULATING JACOBIAN MATRIX TO BE USED TO SOLVE THE INVERSE KINEMATIC
# NUMERICAL APPROACH (DIFFERENTIATION) IS USED
# Running this cell takes about 10 minutes
# Maybe Jump by 2 cells, to load the Jacobian Function from File

# Joint 1 Position vector and rotation axis
H0 = eye(4)
P0 = H0[0:3, -1]
Z0 = H0[0:3, 0]
H0

In [ ]:
# Joint 2 Position vector and rotation axis
H1 = trigsimp(Rx(q1)*Tx(l1))
P1 = H1[0:3, -1]
Z1 = H1[0:3, 1]
H1

In [ ]:
# Joint 3 Position vector and rotation axis
H2 = trigsimp(Rx(q1)*Tx(l1)*Ry(q2)*Tz(l2))
P2 = H2[0:3, -1]
Z2 = H2[0:3, 2]
H2

In [ ]:
# Joint 4 Position vector and rotation axis
H3 = trigsimp(Rx(q1)*Tx(l1)*Ry(q2)*Tz(l2)*Rz(q3)*Tz(l3))
P3 = H3[0:3, -1]
Z3 = H3[0:3, 0]
H3

In [ ]:
# Joint 5 Position vector and rotation axis
H4 = trigsimp(Rx(q1)*Tx(l1)*Ry(q2)*Tz(l2)*Rz(q3)*Tz(l3)*Rx(q4)*Tz(l4))
P4 = H4[0:3, -1]
Z4 = H4[0:3, 2]
H4

In [ ]:
# End Effector Position vector and rotation axis
H5 = trigsimp(Rx(q1)*Tx(l1)*Ry(q2)*Tz(l2)*Rz(q3)*Tz(l3)*Rx(q4)*Tz(l4)*Rz(q5)*Tz(l5))
P5 = H5[0:3, -1]
H5

In [ ]:
J1 = trigsimp(Z0.cross(P5 - P0)).row_insert(len(Z0), Z0)
J2 = trigsimp(Z1.cross(P5 - P1)).row_insert(len(Z1), Z1)
J3 = trigsimp(Z2.cross(P5 - P2)).row_insert(len(Z2), Z2)
J4 = trigsimp(Z3.cross(P5 - P3)).row_insert(len(Z3), Z3)
J5 = trigsimp(Z4.cross(P5 - P4)).row_insert(len(Z4), Z4)

In [ ]:
J_Function = J1.col_insert(1, J2)\
    .col_insert(2, J3)\
    .col_insert(3, J4)\
    .col_insert(4, J5)

In [ ]:
J_Function

In [ ]:
# USING NEWTON RAPHSON TO CALCULATE FOR THE INVERSE

# Desired position and orientation
Xd = [0, 0, 0, 0, 0, 0]

# Current position and orientation
Xc = [0, 0, 0, 0, 0, 0]

# Initial Guess for parameters
q1, q2, q3, q4, q5 = 0, 0, 0, 0, 0

# Jacobian Function
J_Func = lambdify([(q1, q2, q3, q4, q5)], J_Function, "numpy")

# Transformation function
T_Func = lambdify([(q1, q2, q3, q4, q5)], H5, "numpy")

# Transformation matrix of desired position
def D_Transformation(px, py, pz, r, p, y):
    return Matrix([
        [cos(y)*cos(p), cos(y)*sin(p)*sin(r)-sin(y)*cos(r), cos(y)*sin(p)*cos(r)+sin(y)*sin(r), px],
        [sin(y)*cos(p), sin(y)*sin(p)*sin(r)+cos(y)*cos(r), sin(y)*sin(p)*cos(r)-cos(y)*sin(r), py],
        [-sin(p), cos(p)*sin(r), cos(p)*cos(r), pz],
        [0, 0, 0, 1]
    ])

def V_Func(T_Mat):
    twist = H.subs([[T_Mat[0, -1], T_Mat[1, -1], T_Mat[2, -1], T_Mat[2,1], T_Mat[0,2], T_Mat[1,0]]])
    return twist

# List of parameters
thetas = (q1, q2, q3, q4, q5)

DT = D_Transformation(Xd[0], Xd[1], Xd[2], Xd[3], Xd[4], Xd[5])

for _ in range(10):
    thetas = thetas + linalg.pinv(J_Func(thetas)).dot(V_Func(linalg.inv(T_Func(thetas)).dot(DT)))
    print(thetas)

thetas